In [1]:
import torch
import numpy
from transformers import set_seed

In [2]:
import logging
from tqdm import tqdm

In [3]:
def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)

    return logger

In [4]:
set_seed??

# data/model/loss/optimize/init/train

# data

In [6]:
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [25]:
next(iter(train_iter))[0][0]

tensor([ 3,  9,  2,  1,  3,  5, 13,  2,  1, 13,  4, 15,  9,  5,  6,  2,  1, 21,
        19,  1,  9,  1, 18,  1, 17,  2, 12, 12,  8,  5,  3,  9,  2,  1,  3])

In [33]:
' '.join([vocab.to_tokens(i) for i in next(iter(train_iter))[0][0]])

'  m a c h i n e   b y   h   g   w e l l s i t h e   t i m e   t r a v'

In [20]:
vocab.to_tokens(3)

't'

In [35]:
torch.nn.functional.one_hot?

# model 

In [36]:
num_hidden = 256

In [42]:
rnn_layer= torch.nn.RNN?

In [40]:
##embed by one-hot and mlp

In [41]:
rnn_layer = torch.nn.RNN(input_size = len(vocab),hidden_size = num_hidden,
                        nonlinearity='tanh',bias = True,batch_first = False,
                         dropout=0.5,bidirectional=False)

/Users/zhangjiaqi/workingspace/anaconda3/envs/nn_torch/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [44]:
state = torch.zeros((1, batch_size, num_hidden))
state.shape

torch.Size([1, 32, 256])

In [45]:
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

In [49]:
class RNNModel(torch.nn.Module):
    """The RNN model."""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # If the RNN is bidirectional (to be introduced later),
        # `num_directions` should be 2, else it should be 1.
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = torch.nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = torch.nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = torch.nn.functional.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # The fully connected layer will first change the shape of `Y` to
        # (`num_steps` * `batch_size`, `num_hiddens`). Its output shape is
        # (`num_steps` * `batch_size`, `vocab_size`).
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # `nn.GRU` takes a tensor as hidden state
            return torch.zeros((self.num_directions * self.rnn.num_layers,
                                batch_size, self.num_hiddens), device=device)
        else:
            # `nn.LSTM` takes a tuple of hidden states
            return (torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device),
                    torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device))

In [50]:
net = RNNModel(rnn_layer, vocab_size=len(vocab))